In [ ]:
import sklearn
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.linear_model import LinearRegression

In [ ]:
def train_model_m6_lr(hyperparameter, train_df_X, train_df_y):
    """Train a linear regression model with feature selection.

    Selects the K best features from the predictors and fits a linear regression model.

    Args:
        hyperparameter (pd.Series): Hyperparameter values for the model (e.g., number of features).
        train_df_X (pd.DataFrame): Predictor variables for training.
        train_df_y (pd.DataFrame): Target variable for training.

    Returns:
        model (dict): Dictionary containing the fitted feature selector and linear regression model:
            - 'feature_selector': SelectKBest object used to select features.
            - 'regression_model': Trained LinearRegression object.
    """
    
    #UNPACK HYPERPARAMETER
    num_feature = int(hyperparameter['num_features'])
    
    # FEATURE SELECTOR
    def select_features(train_df_X, train_df_y, num_feature):
        ''' Make model to select K best feature. 
            
        Args:
            train_df_X (df) : features matrix for training
            train_df_y (df) : target matrix for training
        
        Returns:
            fs_lr (model) : feature selector
        '''
        
        train_df_y = train_df_y.values.ravel()
        fs_lr = SelectKBest(f_regression, k = num_feature)
        fs_lr.fit(train_df_X, train_df_y)
        
        return fs_lr

    fs_lr = select_features(train_df_X, train_df_y, num_feature)
    
    #TRAIN MODEL
    train_df_X = fs_lr.transform(train_df_X)
    m06_lr = LinearRegression()
    m06_lr.fit(train_df_X, train_df_y)
  
    # PACK MODEL
    model = {"feature_selector": fs_lr, "regression_model": m06_lr}    

    return model

In [ ]:
def produce_forecast_m6_lr(model, train_df_X, test_df_X):
    """Generate forecasts for train and test sets using a trained linear regression model.

    Args:
        model (dict): Contains the trained feature selector and linear regression model:
            - 'feature_selector': SelectKBest object for feature selection.
            - 'regression_model': Fitted LinearRegression object.
        train_df_X (pd.DataFrame): Predictor variables for the training set.
        test_df_X (pd.DataFrame): Predictor variables for the test set.

    Returns:
        train_df_y_hat (np.ndarray): Predicted values for the training set.
        test_df_y_hat (np.ndarray): Predicted values for the test set.
    """
    fs_lr = model['feature_selector']
    m06_lr = model['regression_model']
    
    # SELECT K BEST FEATURES
    train_df_X = fs_lr.transform(train_df_X)
    test_df_X = fs_lr.transform(test_df_X)
    
    # PRODUCE FORECAST
    train_df_y_hat = m06_lr.predict(train_df_X)
    test_df_y_hat = m06_lr.predict(test_df_X)
    
    return train_df_y_hat, test_df_y_hat